In [2]:
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver
import time

In [3]:
driver = webdriver.Chrome()
url = 'https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=cur&date=20221217'
driver.get(url)

In [4]:
result = {}
list_ranking = driver.find_element(By.CLASS_NAME, 'list_ranking')
tds = list_ranking.find_elements(By.CLASS_NAME, 'tit5')

In [5]:
# 영화 제목, 링크 딕셔너리 저장
for td in tds:
    a = td.find_element(By.TAG_NAME, 'a')
    href = a.get_attribute('href')
    title = a.get_attribute('title')
    result[title] = href

In [6]:
# 영화 리스트에 있는 링크들 하나하나 열기
# for key in result.keys():
#     link = result[key]
#     driver.get(link)
#     time.sleep(1)

In [7]:
# 영화 올빼미 창 열기
driver.get(result['올빼미'])

In [8]:
# 평점 창 열기
driver.find_element(By.XPATH, '//*[@id="movieEndTabMenu"]/li[5]').click()

In [9]:
# 리뷰 iframe 창 열기
iframe = driver.find_element(By.ID,'pointAfterListIframe')
frame_src = iframe.get_attribute('src')
driver.get(frame_src)

In [11]:
# 스포일러 보기
spoiler = driver.find_element(By.ID, 'spoilerYnLable').click()